# CV – object detection

В этой тетрадке мы рассмотрим задачу детекции объектов на примере датасета [Stanford Drone Dataset](https://cvgl.stanford.edu/projects/uav_data/)

**Предполагаем, что ноутбук запущен внутри Yandex DataSphere**

In [1]:
from pathlib import Path
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from tqdm import tqdm
import cv2
import os
import torch
import torch.nn as nn
# import torch.nn.functional as F
from torch import optim
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet34
# from torch.utils.tensorboard import SummaryWriter

from collections import Counter, defaultdict
import random

In [2]:
%pip install seaborn
import seaborn as sns

Defaulting to user installation because normal site-packages is not writeable


In [3]:
%pip install albumentations
import albumentations as A
import albumentations.pytorch.transforms as APT

Defaulting to user installation because normal site-packages is not writeable


## Data

Stanford drone dataset – это датасет из нескольких видео, записанных с дрона в восьми местах (`SCENE_NAME`).
Каждое видео покадрово размечено шестью типами объектов (`label`). (Подробное описание датасета лежит в файле `initial_README`)  
В текущей задаче мы будем использовать не исходные видео, а фреймы из них.

Структура директории с датасетом:
- ./stanford-drone-dataset-frames/
    - {train, val}/
        - annotations/
            - {`SCENE_NAME`}/
                - video{`VIDEO_ID`}/
                    - annotations.csv
    - {train, val}/
        - frames/
            - {`SCENE_NAME`}/
                - video{`VIDEO_ID`}/
                    - frame_{`FRAME_IDX`}.jpg

Для каждого файла с аннотацией есть аналогичный путь к директории, в которой находятся фреймы из видео `frame_{FRAME_IDX}.jpg`.  
Например, для фреймов, лежащих в директории `./stanford-drone-dataset-frames/train/frames/quad/video0/`, соответствующий файл с разметкой лежит по пути: `./stanford-drone-dataset-frames/train/annotations/quad/video0/annotations.csv`.

Каждая строка в файлах `annotations.csv` содержит аннотацию одного объекта. Каждый CSV файл содержит 11 колонок с хедером.

Наиболее интересные нам колонки:
- xmin – верхняя левая X-координата бокса объекта (bounding box).
- ymin – верхняя левая Y-координата бокса объекта (bounding box).
- xmax – нижняя правая X-координата бокса объекта (bounding box).
- ymax – нижняя правая Y-координата бокса объекта (bounding box).
- frame_idx – индекс фрейма, который соответсвует текущей строке аннотации.

Сэмпл из одной сцены можно скачать с [Google Drive](https://drive.google.com/file/d/18XeE0kHWqpLyBfZFbAbUTZqbygETTCLC/view?usp=sharing).

In [8]:
# Качаем архив с данными с Yandex Object Storage.

#from cloud_ml.storage.api import Storage

#s3 = Storage.s3(access_key="Le9tg70HQEJsoGqjqXH8", secret_key="NV75mCPkC0PEd35ImyDI5vI7p40YGFOYZgkH7moa")
# downloading contents of the remote file into the local one
#s3.get_dir('dl-hse-2021/stanford-drone-dataset-frames/', './stanford-drone-dataset-frames/')

## Задание 1
**(0.2 балла)** Напишите класс датасет, который будет возвращать картинку и координаты размеченных на ней объектов.

In [5]:
def get_paths (frames_path, annotations_path):
    columns = ['track_id', 'xmin', 'ymin', 'xmax', 'ymax', 'frame', 'lost', 'occluded',
       'generated', 'label', 'frame_idx', 'path']
    df = pd.DataFrame(columns=columns)

    frames_paths = []
    annotations_paths = []
    for i in os.walk(frames_path):
        frames_paths.append(i)
    for i in os.walk(annotations_path):
        annotations_paths.append(i)

    # все пути для кадров    
    frames_paths_ = []   
    for address, dirs, files in frames_paths:
                for file in files:
                    frames_paths_.append(address+'/'+file)

    for address, dirs, files in annotations_paths:
                for file in files:
                    path_ = address+'/'+file
                    df2 = pd.read_csv(path_, dtype={'xmin': int, 'ymin': int, 'xmax': int, 'ymax': int, 'frame_idx' : str})
                    a = address+'/'
                    where = np.where(np.array([a[47:] in frames_paths_[i] for i in range (len(frames_paths_))]) == True)[0]
                    this_frames_paths = [frames_paths_[i] for i in list(where)]
                    list1 = []
                    for i in range (len(list(this_frames_paths))):

                        l = this_frames_paths[i].find('_')
                        r = this_frames_paths[i].find('.')
                        list1.append([this_frames_paths[i][l+1:r], this_frames_paths[i]])
                    this_frames_paths_df = pd.DataFrame(list1, columns = ['frame_idx', 'path'])
                    this_frames_paths_df = this_frames_paths_df.astype({'frame_idx': str})

                    df = df.append(df2.merge(this_frames_paths_df, on = 'frame_idx'))
    return df

In [43]:
df_train = get_paths ('stanford-drone-dataset-frames/train/frames/', 'stanford-drone-dataset-frames/train/annotations/')
df_val = get_paths ('stanford-drone-dataset-frames/val/frames/', 'stanford-drone-dataset-frames/val/annotations/')

In [44]:
df_train = df_train.query('lost != 1')
df_val = df_val.query('lost != 1')

In [45]:
df_train['label'] = df_train['label'].map({'Pedestrian': 1, 'Biker': 2})
df_val['label'] = df_val['label'].map({'Pedestrian': 1, 'Biker': 2})

In [46]:
train_frames_paths = df_train['path'].unique()
val_frames_paths = df_val['path'].unique()

In [11]:
def process_image(image):
    img = np.asarray(image)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32) / 255.0 # img \in [0, 1]
    mean = np.array([0.485, 0.456, 0.406]).reshape(1, 1, 3)
    std =  np.array([0.229, 0.224, 0.225]).reshape(1, 1, 3)
    img = (img - mean) / std
    img = img.astype(np.float32)
       
    return img

In [12]:
class StanfordDroneDataset(Dataset):
    def __init__(self, paths_list, df, transform):
        self.transform = transform
        self.paths_list = paths_list
        self.df = df
        self.transform = transform
        
    def __len__(self):
        return len(self.paths_list)
    
    def __getitem__(self, index):
        img_path = self.paths_list[index]
        img = cv2.imread(img_path)
        img = process_image(img)
        idx = list(np.where(self.df['path'] == self.paths_list[index])[0])
        annot = self.df.iloc[idx,:]
        #annot.loc[annot.loc[:,'ymax'] > img.shape[0], 'ymax'] = img.shape[0]
        #annot.loc[annot.loc[:,'xmax'] > img.shape[1], 'xmax'] = img.shape[1]
        frame_idx = annot['frame_idx'].unique()[0]
        category = list(annot['label'])
        bboxes = list(annot.loc[:,'xmin':'ymax'].itertuples(index=False, name=None))
        ret = {"image": img, "frame_idx": frame_idx, "category": category, "bboxes": bboxes}
        
        if self.transform is not None:
            ret = self.transform(**ret)
        return ret

## Задание 2
(0.2 балла) сделайте подготовку данных на Albumentations, collate_fn и правильный даталоадер, проверьте шейпы выходных тензоров

In [14]:
transform = A.Compose(
    [A.ShiftScaleRotate(p=0.5), A.RandomCrop(720, 720, always_apply=True), # размеры для кропа меньше минимальных размеров кадра, проверила
    APT.ToTensorV2(always_apply=True)],
    bbox_params=A.BboxParams(format='pascal_voc', label_fields=['category']),
)

In [47]:
train_dataset = StanfordDroneDataset(paths_list = train_frames_paths, df = df_train, transform = transform)
val_dataset = StanfordDroneDataset(paths_list = val_frames_paths, df = df_val,  transform = transform)

In [48]:
train_dataset[1]

{'image': tensor([[[-1.5357, -1.5357, -1.5185,  ..., -0.0116, -0.0116,  0.0056],
          [-1.5357, -1.5357, -1.4672,  ...,  0.0056,  0.0227,  0.0227],
          [-1.5357, -1.5357, -1.5870,  ...,  0.0056,  0.0056,  0.0056],
          ...,
          [-0.3198, -0.3198, -0.3027,  ..., -0.1999, -0.2171, -0.2342],
          [-0.3198, -0.3198, -0.3027,  ..., -0.0972, -0.1143, -0.1486],
          [-0.3198, -0.3198, -0.3027,  ..., -0.0629, -0.0801, -0.1143]],
 
         [[-1.5805, -1.5630, -1.5455,  ..., -0.2325, -0.2325, -0.2150],
          [-1.5630, -1.5630, -1.4755,  ..., -0.1800, -0.1625, -0.1625],
          [-1.5630, -1.5630, -1.5980,  ..., -0.1800, -0.1800, -0.1800],
          ...,
          [-0.4601, -0.4601, -0.4426,  ..., -0.4076, -0.4251, -0.4426],
          [-0.4601, -0.4601, -0.4426,  ..., -0.3025, -0.3200, -0.3550],
          [-0.4601, -0.4601, -0.4426,  ..., -0.2675, -0.2850, -0.3200]],
 
         [[-1.1596, -1.1944, -1.1944,  ...,  0.0256,  0.0256,  0.0431],
          [-1.1944,

In [49]:
train_dataset[100]['image'].shape

torch.Size([3, 720, 720])

In [50]:
def collate_fn(lst):
    tmp = defaultdict(list)
    ret = dict()
    for entry in lst:
        for k, v in entry.items():
            tmp[k].append(v)
    
    for k, v in tmp.items():
        if isinstance(v[0], np.ndarray):
            for vv in v:
                print(vv.shape)
            v = np.concatenate(v, 0)
        if isinstance(v[0], torch.Tensor):
            v = torch.cat(v, 0)
        ret[k] = v
    return ret


dl = DataLoader(train_dataset, shuffle=True, batch_size=100, collate_fn=collate_fn)

batch = next(iter(dl))

In [51]:
print(batch['image'][0])
print(batch['frame_idx'][0])
print(batch['category'][0])
print(batch['bboxes'][0])

tensor([[-0.4451, -0.4312, -0.4493,  ...,  0.2282,  0.2250,  0.2111],
        [-0.4498, -0.4556, -0.4616,  ...,  0.2282,  0.2164,  0.2111],
        [-0.4300, -0.4429, -0.4515,  ...,  0.2228,  0.2111,  0.2111],
        ...,
        [-0.1620, -0.1571, -0.1357,  ..., -0.5042, -0.4734, -0.4504],
        [-0.1654, -0.1400, -0.1518,  ..., -0.5135, -0.4798, -0.4573],
        [-0.1528, -0.1389, -0.1673,  ..., -0.5003, -0.4226, -0.4343]])
1370
[1, 1]
[(488.49873019738646, 157.9203529289657, 558.801349015002, 235.90421368847342), (144.04283966914602, 7.084604909225618, 210.6680629451464, 85.99981529225352)]


## Задание 3
(0.4 балла) Приготовьте модель для детекции, проверьте, что все правильно отрабатывает.

## Задание 4
(0.2 балла) натренируйте модель:
- Убедитесь, что она учится,
- Проверьте, что на выходе что-то адекватное.

Трейнер можно взять с любого занятия.
